In [1]:
%env TRANSFORMERS_CACHE=/scratch/alif
%env HF_DATASETS_CACHE=/scratch/alif

env: TRANSFORMERS_CACHE=/scratch/alif
env: HF_DATASETS_CACHE=/scratch/alif


In [2]:
import sys
sys.path.append('..')  # This adds the parent directory to the path

from model import GPT, GPTConfig  # Now you can import your classes

In [3]:
import os
import time
import math
import pickle
from contextlib import nullcontext
import inspect

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

from finetuning.lora import add_lora
from finetuning.utils import tie_weights, get_lora_params, get_lora_state_dict, get_oft_params, get_oft_state_dict
from finetuning.oft import inject_trainable_oft, inject_trainable_oft_conv, inject_trainable_oft_extended, inject_trainable_oft_with_norm

import tiktoken

In [4]:
import torch.nn as nn

In [5]:
from typing import Callable, Dict, List, Optional, Set, Tuple, Type, Union
from oft import OFTInjectedLinear, OFTInjectedConv2d

In [6]:
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False 
always_save_checkpoint = True 

wandb_log = False 
wandb_project = 'owt'
wandb_run_name = 'gpt2' 

dataset = 'openwebtext'
gradient_accumulation_steps = 5 * 8 
batch_size = 12 
block_size = 1024

n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 
bias = False 

learning_rate = 6e-4 
max_iters = 600000 
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 

decay_lr = True 
warmup_iters = 2000 
lr_decay_iters = 600000 
min_lr = 6e-5 

backend = 'nccl' 
device = 'cuda' 
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' 
compile = True 

init_from = 'gpt2-large'

model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) 
override_args = dict(dropout=dropout)

In [7]:
def unique_modules(model):
    module_names = []
    for module in model.modules():
        name = module.__class__.__name__

        if name not in module_names:
            module_names.append(name)
            print(f'module: \n{module}\n\n')
        
    return module_names

In [8]:
def _find_modules(
    model,
    ancestor_class: Optional[Set[str]] = None,
    search_class: List[Type[nn.Module]] = [nn.Linear],
    exclude_children_of: Optional[List[Type[nn.Module]]] = [
        OFTInjectedLinear,
        OFTInjectedConv2d,
    ],
):
    """
    Find all modules of a certain class (or union of classes) that are direct or
    indirect descendants of other modules of a certain class (or union of classes).
    Returns all matching modules, along with the parent of those moduless and the
    names they are referenced by.
    """

    # Get the targets we should replace all linears under
    if ancestor_class is not None:
        ancestors = (
            module
            for module in model.modules()
            if module.__class__.__name__ in ancestor_class
        )
    else:
        # the first modules is the most senior father class.
        # this, incase you want to naively iterate over all modules.
        for module in model.modules():
            ancestor_class = module.__class__.__name__
            break
        ancestors = (
            module
            for module in model.modules()
            if module.__class__.__name__ in ancestor_class
        )

    results = []
    # For each target find every linear_class module that isn't a child of a OFTInjectedLinear
    for ancestor in ancestors:
        for fullname, module in ancestor.named_modules():
            if any([isinstance(module, _class) for _class in search_class]):
                # Find the direct parent if this is a descendant, not a child, of target
                *path, name = fullname.split(".")
                parent = ancestor
                while path:
                    parent = parent.get_submodule(path.pop(0))
                # Skip this linear if it's a child of a OFTInjectedLinear
                if exclude_children_of and any(
                    [isinstance(parent, _class) for _class in exclude_children_of]
                ):
                    continue
                results.append((parent, name, module))  # Append the result to the list

    return results  # Return the list instead of using 'yield'

In [9]:
def inject_oft(model):
    target_replace_module = ["CausalSelfAttention"]
    verbose = False
    r = 4
    eps = 1e-5
    is_coft = True
    block_share = False

    collected_modules = []
    module_count = 0

    collected_dict = {}
    names = []
    require_grad_params = []

    for _module, name, _child_module in _find_modules(
            model, target_replace_module, search_class=[nn.Linear]
        ):

        if name not in collected_modules: 
            collected_modules.append(name)
            collected_dict.update({name: 1})

            print(f'Module name: {name}')
            print(f'Module: {_module}')
            print(f'Child module: {_child_module}\n\n')

        collected_dict[name] += 1
        module_count += 1

        weight = _child_module.weight
        bias = _child_module.bias

        if verbose:
            print("OFT Injection : injecting oft into ", name)
            print("OFT Injection : weight shape", weight.shape)

        _tmp = OFTInjectedLinear(
            _child_module.in_features,
            _child_module.out_features,
            _child_module.bias is not None,
            r=r,
            eps=eps,
            is_coft=is_coft,
            block_share=block_share,
        )
        _tmp.OFT.weight = weight
        if bias is not None:
            _tmp.OFT.bias = bias

        _tmp.to(_child_module.weight.device).to(_child_module.weight.dtype)
        _module._modules[name] = _tmp

        require_grad_params.append(_module._modules[name].R)
        _module._modules[name].R.requires_grad = True
        names.append(name)
    
    
    for key, value in collected_dict.items():
        print(f"Collected {value} '{key}' modules")

In [10]:
def get_oft_params(model, print_shapes=False):
    return get_params_by_name(model, print_shapes=print_shapes, name_filter=name_is_oft)

def get_params_by_name(model, print_shapes=False, name_filter=None):
    for n, p in model.named_parameters():
        if name_filter is None or name_filter(n):
            if print_shapes:
                print(n, p.shape)
            yield p

def name_is_oft(name):
    return ("OFT" in name)

In [12]:
model = GPT.from_pretrained(init_from, override_args)
for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
    model_args[k] = getattr(model.config, k)

model_type:  gpt2-large


/home/alif/oft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading weights from pre-trained GPT: gpt2-large
Forcing vocab_size=50257, block_size=1024, bias=True (GPT checkpoint pre-reqs)
Overriding dropout rate to 0.0


In [13]:
module_names = unique_modules(model)

module: 
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (dropout): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-35): 36 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1280, out_features=3840, bias=True)
          (c_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)


module: 
ModuleDict(
  (wte): Embedding(5025

In [14]:
module_names

['GPT',
 'ModuleDict',
 'Embedding',
 'Dropout',
 'ModuleList',
 'Block',
 'LayerNorm',
 'CausalSelfAttention',
 'Linear',
 'MLP',
 'GELU']

In [15]:
from oft import OFTInjectedLinear, OFTInjectedConv2d, inject_trainable_oft

In [16]:
model.requires_grad_(False)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (dropout): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-35): 36 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1280, out_features=3840, bias=True)
          (c_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [17]:
oft_modules = ["CausalSelfAttention"]
device_type = 'cuda'

scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
model.to(device)
oft_params, train_names = inject_trainable_oft(model, target_replace_module=oft_modules, verbose=False, r=4, eps=1e-3, is_coft=True, block_share=False)
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

In [67]:

print(len(oft_params))

param_count = 0
for param in oft_params:
    if param.requires_grad:
        param_count += param.numel()
        
print(f'param count: {param_count:,}')

72
param count: 29,491,200


In [37]:
search_class = [nn.Linear]
results = []
exclude_children_of = None

for elem in model.modules():
    module_name = elem.__class__.__name__
    if module_name in oft_modules:
        ancestor = elem
        
        for fullname, module in ancestor.named_modules():
            if any([isinstance(module, _class) for _class in search_class]):
                
                *path, name = fullname.split(".")
                parent = ancestor
                while path:
                    parent = parent.get_submodule(path.pop(0))
                # Skip this linear if it's a child of a OFTInjectedLinear
                if exclude_children_of and any(
                    [isinstance(parent, _class) for _class in exclude_children_of]
                ):
                    continue
                results.append((parent, name, module))

In [39]:
results[0]

(OFTInjectedLinear(
   (OFT): Linear(in_features=1280, out_features=3840, bias=True)
 ),
 'OFT',
 Linear(in_features=1280, out_features=3840, bias=True))

In [59]:
oft_linear = results[0][2]
oft_linear.weight

Parameter containing:
tensor([[ 0.1655, -0.2344,  0.1063,  ...,  0.0020, -0.1146,  0.0222],
        [ 0.1230,  0.1413, -0.0397,  ...,  0.1257, -0.0897, -0.0171],
        [ 0.1003,  0.0706,  0.1085,  ..., -0.0798, -0.0925, -0.0463],
        ...,
        [-0.0081, -0.0105, -0.0042,  ...,  0.0024,  0.0013,  0.0290],
        [ 0.0106,  0.0239,  0.0183,  ...,  0.0351,  0.0007,  0.0258],
        [-0.0183, -0.0101, -0.0080,  ...,  0.0204, -0.0041, -0.0327]],
       device='cuda:0')

In [65]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def get_lr(it):

    if it < warmup_iters:
        return learning_rate * it / warmup_iters

    if it > lr_decay_iters:
        return min_lr

    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) 
    return min_lr + coeff * (learning_rate - min_lr)

In [66]:
ddp = int(os.environ.get('RANK', -1)) != -1 
if ddp:
    print("Initializing distributed training...")
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 
    seed_offset = ddp_rank 
    assert gradient_accumulation_steps % ddp_world_size == 0
    gradient_accumulation_steps //= ddp_world_size
else:
    print("Training on 1 GPU...")
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

Training on 1 GPU...
tokens per iteration will be: 491,520


In [67]:
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

data_dir = os.path.join('../data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':

        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

iter_num = 0
best_val_loss = 1e9

meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

In [ ]:
X, Y = get_batch('train') 
t0 = time.time()
local_iter_num = 0 
raw_model = model.module if ddp else model 
running_mfu = -1.0
enc = tiktoken.get_encoding("gpt2")

while True:

    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # sample_generation = model.generate(X, max_new_tokens=100)
        # decoded_generation = enc.decode(sample_generation[0].tolist())
        # print(f'\n\nSample Generation: \n{decoded_generation}\n\n')
        
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                if use_lora:
                    checkpoint['lora'] = get_lora_state_dict(raw_model)
                
                print(f"saving checkpoint to {out_dir}")
                checkpoint_name = dataset + '_' +  init_from + '_' + 'ckpt.pt'
                torch.save(checkpoint, os.path.join(out_dir, checkpoint_name))
    if iter_num == 0 and eval_only:
        break

    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps 

        X, Y = get_batch('train')
        scaler.scale(loss).backward()

    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)

    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: 
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    if iter_num > max_iters:
        break